In [2]:
import sys
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import *
import numpy as np
import h5py
import matplotlib.pyplot as plt
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.initializers import glorot_uniform

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}") 
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(tf.config.list_physical_devices('GPU'))


Tensor Flow Version: 2.11.0
Keras Version: 2.11.0

Python 3.9.10 (tags/v3.9.10:f2f3f53, Jan 17 2022, 15:14:21) [MSC v.1929 64 bit (AMD64)]
Pandas 1.5.3
Scikit-Learn 1.2.1
[]


In [3]:
num_classes = 7
img_rows,img_cols = 48,48
batch_size = 32
epochs = 25

with open(r"C:\Users\admin\Desktop\Major_6700\fer2013.csv") as f:
  content = f.readlines()

lines = np.array(content)

num_of_instances = lines.size
print("number of instances: ",num_of_instances)
print("instance length: ",len(lines[1].split(",")[1].split(" ")))

#------------------------------
#initialize trainset and test set
x_train, y_train, x_test, y_test = [], [], [], []

#------------------------------
#transfer train and test set data
for i in range(1,num_of_instances):
    try:
        emotion, img, usage = lines[i].split(",")
          
        val = img.split(" ")
            
        pixels = np.array(val, 'float32')
        
        emotion = to_categorical(emotion, num_classes)
    
        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
    except:
      print("", end="")

#------------------------------
#data transformation for train and test sets
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

x_train /= 255 #normalize inputs between [0, 1]
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

number of instances:  35888
instance length:  2304
28709 train samples
3589 test samples


In [4]:
def identity_block(X, f, filters, stage, block):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_bname_base  = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3= filters
    
    X_shortcut = X
    
    X = Conv2D(F1, (1,1), (1,1), padding= 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform())(X)
    X = BatchNormalization(axis = -1, name = bn_bname_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F2, (f, f), (1, 1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform())(X)
    X = BatchNormalization(axis = -1, name = bn_bname_base + '2b')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F3, (1, 1), (1, 1), padding ='valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform())(X)
    X = BatchNormalization(axis = -1, name = bn_bname_base + '2c')(X)
    
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    
    return X

In [5]:
def convolutional_block(X, f, filters, stage, block, s =2):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform())(X)
    X = BatchNormalization(axis = -1, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F2, (f,f), strides=(1,1), padding='same', name=conv_name_base+'2b', kernel_initializer = glorot_uniform())(X)
    X = BatchNormalization(axis=-1, name = bn_name_base+'2b')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F3, (1,1), strides = (1,1), padding = 'valid', name = conv_name_base+'2c', kernel_initializer=glorot_uniform())(X)
    X = BatchNormalization(axis=-1, name = bn_name_base + '2c')(X)
    
    X_shortcut = Conv2D(F3, (1,1), strides=(s,s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform())(X_shortcut)
    X_shortcut = BatchNormalization(axis=-1, name=bn_name_base+'1')(X_shortcut)
    
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    
    return X


In [6]:
def ResNet50(input_shape, classes):
    x_input = Input(input_shape)
    
    X = ZeroPadding2D((3,3))(x_input)
    
    X = Conv2D(64, (3,3), (1,1), name='conv1', kernel_initializer=glorot_uniform())(X)
    X = BatchNormalization(axis=-1, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2))(X)
    
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    X = convolutional_block(X, f=3, filters = [128,128,512], stage = 3, s=2, block = 'a')
    X = identity_block(X, 3, [128,128,512], stage = 3,block ='b')
    X = identity_block(X, 3, [128,128,512], stage = 3,block ='c')
    X = identity_block(X, 3, [128,128,512], stage = 3,block ='d')

    X = convolutional_block(X,  3, [256,256,1024], s=2, stage = 4,block = 'a')
    X = identity_block(X, 3, [256,256,1024], stage = 4,block='b')
    X = identity_block(X, 3, [256,256,1024], stage = 4,block='c')
    X = identity_block(X, 3, [256,256,1024], stage = 4,block='d')
    X = identity_block(X, 3, [256,256,1024], stage = 4,block='e')
    X = identity_block(X, 3, [256,256,1024], stage = 4,block='f')

    
    X = convolutional_block(X, 3, [512,512,2048], s = 2,stage = 5,block='a')
    X = identity_block(X, 3, [512,512,2048], stage = 5,block = 'b')
    X = identity_block(X, 3, [512,512,2048], stage = 5,block = 'c')
    
    X = AveragePooling2D((2,2), name = 'avg_pool')(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name = 'fc' + str(classes), kernel_initializer=glorot_uniform())(X)

    model = Model(inputs = x_input, outputs = X, name = 'ResNet50')
    return model

In [7]:
model = ResNet50((48,48,1), num_classes)

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
print ("number of training examples = " + str(x_train.shape[0]))
print ("number of test examples = " + str(x_test.shape[0]))
print ("X_train shape: " + str(x_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(x_test.shape))
print ("Y_test shape: " + str(y_test.shape))

number of training examples = 28709
number of test examples = 3589
X_train shape: (28709, 48, 48, 1)
Y_train shape: (28709, 7)
X_test shape: (3589, 48, 48, 1)
Y_test shape: (3589, 7)


In [10]:
model.fit(x_train, y_train, epochs = 100, batch_size = 32) #model has been trained for a total of 20 epochs

Epoch 1/100
898/898 [==============================] - 1973s 2s/step - loss: 2.2431 - accuracy: 0.2534
Epoch 2/100
898/898 [==============================] - 1933s 2s/step - loss: 1.7025 - accuracy: 0.3878
Epoch 3/100
898/898 [==============================] - 1894s 2s/step - loss: 1.6014 - accuracy: 0.4268
Epoch 4/100
898/898 [==============================] - 1888s 2s/step - loss: 1.4975 - accuracy: 0.4448
Epoch 5/100
898/898 [==============================] - 1889s 2s/step - loss: 1.7571 - accuracy: 0.3512
Epoch 6/100
898/898 [==============================] - 1882s 2s/step - loss: 1.4020 - accuracy: 0.4816
Epoch 7/100
898/898 [==============================] - 1880s 2s/step - loss: 1.3238 - accuracy: 0.5000
Epoch 8/100
898/898 [==============================] - 1879s 2s/step - loss: 1.4213 - accuracy: 0.4704
Epoch 9/100
898/898 [==============================] - 1880s 2s/step - loss: 1.2033 - accuracy: 0.5450
Epoch 10/100
898/898 [==============================] - 1875s 2s/step - l

In [14]:
model=model.save('ResNet50.h5')